In [ ]:
!voila test_chatbot.ipynb

In [ ]:
#wigets and display widgets
import ipywidgets as wg
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display

#NLP and clustering
import spacy
from clustering import Clustering
from topic_determination import Topic_determination

In [ ]:
#!pip install azure-ai-textanalytics

In [ ]:
#!python -m spacy download de_core_news_lg

In [ ]:
nlp = spacy.load('de_core_news_lg')
clustering = Clustering(nlp)
topic_determination = Topic_determination()
metadata = []
topics = []
blacklist = {}
df = None

In [ ]:
wg.HTML(
    value="Lieber <b>Nutzer</b> bitte stelle deine Frage:",
    placeholder='',
    description='',
)

In [ ]:
def best_topic_to_question(x):
    global topics
    global blacklist
    global df
    global metadata
    if x == "ja":
        print("\n".join(df.loc[[topics[0] in topic for topic in df['Topics']]]["d115Name"].tolist()))
        metadata = []
        topics = []
        blacklist = {}
        return 
    elif x == "nein":
        blacklist.update({topics[0]:True})
        topics.pop(0)
        if topics:
            return "Geht es bei ihrem Anliegen um " + topics[0] + "?"
        metadata = []
        topics = []
        blacklist = {}
        return "Sorry nothing there."
    else:
        topic_count ={}
        metadata.append(x)
        df = clustering.run("antrag", cluster_by="ssdsLemma", do_preprocessing=True, query_max_elems=2000, cluster_eps=1.3, cluster_min_samples=1)
        #df = clustering.run(" ".join(metadata), cluster_by="ssdsLemma", do_preprocessing=True, query_max_elems=2000, cluster_eps=1.3, cluster_min_samples=1)
    
        df_res=topic_determination.run(df,1,"ssdsLemma","c09dde90c9f34da3b69065aabdb735c8")
        for topiclist in df['Topics']:
            for topic in topiclist:
                if topic not in blacklist:
                    topic_count.update({topic:topic_count.get(topic, 0)+1})
                    topics.append(topic)
        topics = list(set(topics))
        topics.sort(key=lambda x: topic_count[x], reverse=True)
        return "Geht es bei ihrem Anliegen um " + topics[0] + "?"

In [ ]:
interact(lambda Frage: best_topic_to_question(Frage), Frage="Antrag");

In [ ]:
#name = wg.Text("Hello")
#question = wg.Text("antrag")
# widgets.HTML(
#     value="Hello <b>User</b> please enter your questions here!",
#     placeholder='',
#     description='',
# )
#display(question)

In [ ]:
#widgetLink = widgets.jslink((floatTextBox, 'value'), (floatSlider, 'value'))

In [ ]:
# widgets.Textarea(
#     value='Hello World',
#     placeholder='Type something',
#     description='String:',
#     disabled=False
# )